In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import os
import boto3
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import csv
import numpy as np

In [3]:
def maybe_pickle(data, filename, force=False):
    if os.path.exists(filename) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping pickling.' % filename)
    else:
        print('Pickling %s.' % filename)
        try:
            with open(filename, 'wb') as f:
                pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', filename, ':', e)


def maybe_download(table, force=False):
    storage_file = table + ".pkl"
    if force or not os.path.exists(storage_file):
        data = data_retrieve(table)
        maybe_pickle(data, storage_file)
    return storage_file
        
def data_retrieve(table, page_size=100):
    """download data from table"""
    client = boto3.client('dynamodb')
    paginator = client.get_paginator('scan')

    counter = 0
    items = []
    for page in paginator.paginate(TableName=table, PaginationConfig={"PageSize": page_size}):
        items.extend(page["Items"])
        
        counter += 1
        if counter % 50 == 0:  # print debug every 50 pages
            print("items: {}. next page. {}. Count {}. ScannedCount: {}".format(len(items), counter, page["Count"], page["ScannedCount"]))


    return items
    

In [4]:
data_file = maybe_download("apthunt")
data = pickle.load(open(data_file, 'rb'))

In [5]:
print("wow")

wow


In [6]:
# To go from dynamo format to python
from boto3.dynamodb import types

deserializer = types.TypeDeserializer()
python_data = [{k: deserializer.deserialize(v) for k,v in d.items()} for d in data]

In [7]:
del data

In [8]:
df = pd.DataFrame(python_data)

In [9]:
del python_data

In [10]:
with open("df.pkl", 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)


In [11]:
df = pickle.load(open("df.pkl", 'rb'))

In [12]:
df.head()

,parsed_postingtitletext,parsed_map_address,added,PostUrl,PostTitle,parsed_price,parsed_price_text,parsed_housing,PostContent,parsed_data_longitude,...,parsed_postingbody,parsed_page_head,parsed_thumbs,FeedTitle,parsed_titletextonly,intid,parsed_data_latitude,FeedUrl,PostPublished,parsed_notices
0,$3150 / 2br - 1300ft2 - Sunset Oaks Townhome- ...,1249 Crescent Terrace,1538706586266,https://sfbay.craigslist.org/sby/apa/d/sunset-...,simulated,3150,$3150,2br - 1300ft2,simulated,-122.026428,...,"Well designed and spacious 2 bedroom, 2 1/2 ba...",None,[https://images.craigslist.org/00Z0Z_5KZmIXxAc...,simulated,Sunset Oaks Townhome- 1249 Crescent Terrace - ...,6aea5603583644368e76bac8293aa38a,37.353754,simulated,simulated,[do NOT contact me with unsolicited services o...
1,$1769 / 2br - 875ft2 - ❄ Pittsburg Apartment L...,2205 E Leland Rd,1539451812116,https://sfbay.craigslist.org/eby/apa/d/pittsbu...,simulated,1769,$1769,2br - 875ft2,simulated,-121.864673,...,Diamond Hillside Apartment Homes offers spacio...,None,[https://images.craigslist.org/00c0c_505E4utDZ...,simulated,"❄ Pittsburg Apartment Living, Gated Community,...",0f490163255448ac973218ae45b7fa99,38.008321,simulated,simulated,[do NOT contact me with unsolicited services o...
2,$3000 / 2br - 1800ft2 - 2 Bedroom 1 Bath 1 Car...,375 Pope Street,1538947757137,https://sfbay.craigslist.org/sfc/apa/d/2-bedro...,simulated,3000,$3000,2br - 1800ft2,simulated,-122.439706,...,clean 2 Bedroom 1 full bathroom 1 car garage. ...,None,[https://images.craigslist.org/00G0G_2bsd96Q2m...,simulated,2 Bedroom 1 Bath 1 Car Garage House,ef4a3ac21bca461bab5364dbda00144c,37.710526,simulated,simulated,[do NOT contact me with unsolicited services o...
3,$2150 / 3br - 1200ft2 - Updated 1200 SQ FT 3 B...,2436 Baltic Dr,1585287116209,https://sfbay.craigslist.org/eby/apa/d/fairfie...,simulated,2150,$2150,3br - 1200ft2,simulated,-122.019156,...,Updated 1200 SQ FT 3 Bed 1.5 Bath 2-story hous...,None,[https://images.craigslist.org/00s0s_fp0FHrerE...,simulated,Updated 1200 SQ FT 3 Bed 1.5 Bath 2-story hous...,bad01973976c49a88c0f45894ae19cc3,38.27479,simulated,simulated,[do NOT contact me with unsolicited services o...
4,$2150 / 1br - 726ft2 - FULLY RENOVATED 1BED 1B...,1182 E Street near 2nd,1587684611249,https://sfbay.craigslist.org/eby/apa/d/hayward...,simulated,2150,$2150,1br - 726ft2,simulated,-122.076737,...,Beautiful Fully Renovated 1 bedroom 1 bathroom...,None,[https://images.craigslist.org/01010_IJCcnFKWL...,simulated,FULLY RENOVATED 1BED 1BATH IN HAYWARD! PRE-LEA...,51f6a9ed9e224966b5fe96faf83521a1,37.671644,simulated,simulated,[do NOT contact me with unsolicited services o...


In [13]:
df["parsed_page_head"].describe()

count                                                  5856
unique                                                 5049
top       "FULLY REMODELED" 3 Bed, 1 Bath, Duplex - apts...
freq                                                     15
Name: parsed_page_head, dtype: object

In [14]:
# small cleanup of "simulated"
del df["FeedTitle"]
del df["FeedUrl"]
del df["PostContent"]
del df["PostPublished"]
del df["PostTitle"]
# delete parsed_notices. looks like it is the same
del df["parsed_notices"]
# no need for price text
del df["parsed_price_text"]

In [15]:
df.columns

Index(['parsed_postingtitletext', 'parsed_map_address', 'added', 'PostUrl',
       'parsed_price', 'parsed_housing', 'parsed_data_longitude',
       'parsed_attrs', 'parsed_district', 'parsed_map_link',
       'parsed_postingbody', 'parsed_page_head', 'parsed_thumbs',
       'parsed_titletextonly', 'intid', 'parsed_data_latitude'],
      dtype='object')

In [16]:
# for col in df.columns:
#     print(col)
#     print(df[col].describe())
#     print(df[col].head())
#     print("=" * 60)

In [17]:
def clean_parsed_args(args):
    if not isinstance(args, list):
        return []
    clean = []
    for p in args:
        # remove days
        if p.startswith("friday") or p.startswith("monday") or p.startswith("saturday") or p.startswith("sunday") or p.startswith("thursday") or p.startswith("tuesday") or p.startswith("wednesday"):
            continue
        # remove br/ba
        if "BR / " in p: continue
        # remove ft2
        if p.endswith("ft2"): continue

        # remove application fee
        if p.startswith("application fee"): continue
        
        # remove broker fee
        if p.startswith("broker fee"): continue
        
        # remove available
        if p.startswith("available"): continue
        
        # remove listed by
        if p.startswith("listed by"): continue
        clean.append(p)
    return clean
        

In [19]:
df['parsed_attrs'] = df['parsed_attrs'].map(clean_parsed_args)
# one_hot = pd.get_dummies(df['parsed_attrs'])

In [20]:
df['parsed_attrs']

0         [townhouse, w/d in unit, no smoking, detached ...
1                                               [apartment]
2         [house, laundry in bldg, no smoking, attached ...
3                     [house, w/d hookups, detached garage]
4         [cats are OK - purrr, dogs are OK - wooof, apa...
                                ...                        
843726    [cats are OK - purrr, apartment, laundry on si...
843727    [townhouse, w/d in unit, no smoking, detached ...
843728                                                   []
843729    [cats are OK - purrr, dogs are OK - wooof, fur...
843730        [apartment, laundry on site, detached garage]
Name: parsed_attrs, Length: 843731, dtype: object

In [21]:
df.head()

,parsed_postingtitletext,parsed_map_address,added,PostUrl,parsed_price,parsed_housing,parsed_data_longitude,parsed_attrs,parsed_district,parsed_map_link,parsed_postingbody,parsed_page_head,parsed_thumbs,parsed_titletextonly,intid,parsed_data_latitude
0,$3150 / 2br - 1300ft2 - Sunset Oaks Townhome- ...,1249 Crescent Terrace,1538706586266,https://sfbay.craigslist.org/sby/apa/d/sunset-...,3150,2br - 1300ft2,-122.026428,"[townhouse, w/d in unit, no smoking, detached ...",sunnyvale,https://maps.google.com/?q=loc%3A+%31%32%34%39...,"Well designed and spacious 2 bedroom, 2 1/2 ba...",None,[https://images.craigslist.org/00Z0Z_5KZmIXxAc...,Sunset Oaks Townhome- 1249 Crescent Terrace - ...,6aea5603583644368e76bac8293aa38a,37.353754
1,$1769 / 2br - 875ft2 - ❄ Pittsburg Apartment L...,2205 E Leland Rd,1539451812116,https://sfbay.craigslist.org/eby/apa/d/pittsbu...,1769,2br - 875ft2,-121.864673,[apartment],"Pittsburg, CA",https://maps.google.com/?q=loc%3A+%32%32%30%35...,Diamond Hillside Apartment Homes offers spacio...,None,[https://images.craigslist.org/00c0c_505E4utDZ...,"❄ Pittsburg Apartment Living, Gated Community,...",0f490163255448ac973218ae45b7fa99,38.008321
2,$3000 / 2br - 1800ft2 - 2 Bedroom 1 Bath 1 Car...,375 Pope Street,1538947757137,https://sfbay.craigslist.org/sfc/apa/d/2-bedro...,3000,2br - 1800ft2,-122.439706,"[house, laundry in bldg, no smoking, attached ...",excelsior / outer mission,https://maps.google.com/?q=loc%3A+%33%37%35+Po...,clean 2 Bedroom 1 full bathroom 1 car garage. ...,None,[https://images.craigslist.org/00G0G_2bsd96Q2m...,2 Bedroom 1 Bath 1 Car Garage House,ef4a3ac21bca461bab5364dbda00144c,37.710526
3,$2150 / 3br - 1200ft2 - Updated 1200 SQ FT 3 B...,2436 Baltic Dr,1585287116209,https://sfbay.craigslist.org/eby/apa/d/fairfie...,2150,3br - 1200ft2,-122.019156,"[house, w/d hookups, detached garage]",fairfield / vacaville,"https://www.google.com/maps/search/38.274790,-...",Updated 1200 SQ FT 3 Bed 1.5 Bath 2-story hous...,None,[https://images.craigslist.org/00s0s_fp0FHrerE...,Updated 1200 SQ FT 3 Bed 1.5 Bath 2-story hous...,bad01973976c49a88c0f45894ae19cc3,38.27479
4,$2150 / 1br - 726ft2 - FULLY RENOVATED 1BED 1B...,1182 E Street near 2nd,1587684611249,https://sfbay.craigslist.org/eby/apa/d/hayward...,2150,1br - 726ft2,-122.076737,"[cats are OK - purrr, dogs are OK - wooof, apa...",hayward / castro valley,"https://www.google.com/maps/search/37.671644,-...",Beautiful Fully Renovated 1 bedroom 1 bathroom...,None,[https://images.craigslist.org/01010_IJCcnFKWL...,FULLY RENOVATED 1BED 1BATH IN HAYWARD! PRE-LEA...,51f6a9ed9e224966b5fe96faf83521a1,37.671644


In [22]:
df.shape

(843731, 16)

In [23]:
def get_type(attrs):
    if not isinstance(attrs, list):
        return None
    types = set([
        "apartment", 
        "townhouse", 
        "loft", 
        "land", 
        "house", 
        "duplex", 
        "flat", 
        "condo", 
        "cottage/cabin"
    ])
    return ",".join(sorted(types & set(attrs)))

In [59]:
def prepare_data(df):
    new  = pd.DataFrame()
    new["latitude"] = df["parsed_data_latitude"]
    new["longitude"] = df["parsed_data_longitude"]
    new["district"] = df["parsed_district"].map(lambda s: s.strip("()") if isinstance(s, str) else s)
    new["address"] = df["parsed_map_address"].map(lambda x: x if x != "(google map)" else None)
    new["housing"] = df["parsed_housing"]
    
    new["price"] = df["parsed_price"].map(lambda x: float(x) if x else x)
    
    # to numerical values
    new['price'] = pd.to_numeric(new['price'])
    new['longitude'] = pd.to_numeric(new['longitude'])
    new['latitude'] = pd.to_numeric(new['latitude'])
    
    new['bedrooms'] = new['housing'].str.extract('(\d+)br\s.*', expand=True)
    new['bedrooms'] = pd.to_numeric(new['bedrooms'])
    
    new['area'] = new['housing'].str.extract('(\d+)ft2', expand=True)
    new['area'] = pd.to_numeric(new['area'])
    
    new['type'] = df["parsed_attrs"].map(get_type)
    new['catsok'] = df["parsed_attrs"].map(lambda x: "cats are OK - purrr" in x)
    new['dogsok'] = df["parsed_attrs"].map(lambda x: "dogs are OK - wooof" in x)
    new['garagea'] = df["parsed_attrs"].map(lambda x: "attached garage" in x)
    new['garaged'] = df["parsed_attrs"].map(lambda x: "detached garage" in x)
    new['furnished'] = df["parsed_attrs"].map(lambda x: "furnished" in x)
    
    new['laundryb'] = df["parsed_attrs"].map(lambda x: "laundry in bldg" in x)
    new['laundrys'] = df["parsed_attrs"].map(lambda x: "laundry on site" in x)
    
    new['wd'] = df["parsed_attrs"].map(lambda x: "w/d in unit" in x)
    new['nthumbs'] = df["parsed_thumbs"].apply(lambda x: len(x) if isinstance(x, list) else 0)

    return new

In [60]:
short = prepare_data(df)

In [61]:
short

,latitude,longitude,district,address,housing,price,bedrooms,area,type,catsok,dogsok,garagea,garaged,furnished,laundryb,laundrys,wd,nthumbs
0,37.353754,-122.026428,sunnyvale,1249 Crescent Terrace,2br - 1300ft2,3150.0,2.0,1300.0,townhouse,False,False,False,True,False,False,False,True,18
1,38.008321,-121.864673,"Pittsburg, CA",2205 E Leland Rd,2br - 875ft2,1769.0,2.0,875.0,apartment,False,False,False,False,False,False,False,False,8
2,37.710526,-122.439706,excelsior / outer mission,375 Pope Street,2br - 1800ft2,3000.0,2.0,1800.0,house,False,False,True,False,False,True,False,False,5
3,38.274790,-122.019156,fairfield / vacaville,2436 Baltic Dr,3br - 1200ft2,2150.0,3.0,1200.0,house,False,False,False,True,False,False,False,False,10
4,37.671644,-122.076737,hayward / castro valley,1182 E Street near 2nd,1br - 726ft2,2150.0,1.0,726.0,apartment,True,True,False,False,False,False,False,True,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843726,37.337841,-121.978369,santa clara,2865 Homestead Rd,1br - 750ft2,2195.0,1.0,750.0,apartment,True,False,False,False,False,False,True,False,12
843727,37.552033,-121.996875,fremont / union city / newark,4095 EGGERS DR,2br - 1000ft2,2800.0,2.0,1000.0,townhouse,False,False,False,True,False,False,False,True,8
843728,NaN,NaN,fairfield / vacaville,NaN,/ 2br - 918ft2 -,1965.0,2.0,918.0,,False,False,False,False,False,False,False,False,18
843729,37.787295,-122.392719,Rincon Hill,"340 Fremont St, San Francisco, CA 94105, USA",1br - 708ft2,4890.0,1.0,708.0,apartment,True,True,False,False,True,False,False,True,22


In [62]:
short.shape

(843731, 18)

In [63]:
# del short["housing"]
# del short["address"]

In [64]:
short.dtypes

latitude     float64
longitude    float64
district      object
address       object
housing       object
price        float64
bedrooms     float64
area         float64
type          object
catsok          bool
dogsok          bool
garagea         bool
garaged         bool
furnished       bool
laundryb        bool
laundrys        bool
wd              bool
nthumbs        int64
dtype: object

In [65]:
short

,latitude,longitude,district,address,housing,price,bedrooms,area,type,catsok,dogsok,garagea,garaged,furnished,laundryb,laundrys,wd,nthumbs
0,37.353754,-122.026428,sunnyvale,1249 Crescent Terrace,2br - 1300ft2,3150.0,2.0,1300.0,townhouse,False,False,False,True,False,False,False,True,18
1,38.008321,-121.864673,"Pittsburg, CA",2205 E Leland Rd,2br - 875ft2,1769.0,2.0,875.0,apartment,False,False,False,False,False,False,False,False,8
2,37.710526,-122.439706,excelsior / outer mission,375 Pope Street,2br - 1800ft2,3000.0,2.0,1800.0,house,False,False,True,False,False,True,False,False,5
3,38.274790,-122.019156,fairfield / vacaville,2436 Baltic Dr,3br - 1200ft2,2150.0,3.0,1200.0,house,False,False,False,True,False,False,False,False,10
4,37.671644,-122.076737,hayward / castro valley,1182 E Street near 2nd,1br - 726ft2,2150.0,1.0,726.0,apartment,True,True,False,False,False,False,False,True,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843726,37.337841,-121.978369,santa clara,2865 Homestead Rd,1br - 750ft2,2195.0,1.0,750.0,apartment,True,False,False,False,False,False,True,False,12
843727,37.552033,-121.996875,fremont / union city / newark,4095 EGGERS DR,2br - 1000ft2,2800.0,2.0,1000.0,townhouse,False,False,False,True,False,False,False,True,8
843728,NaN,NaN,fairfield / vacaville,NaN,/ 2br - 918ft2 -,1965.0,2.0,918.0,,False,False,False,False,False,False,False,False,18
843729,37.787295,-122.392719,Rincon Hill,"340 Fremont St, San Francisco, CA 94105, USA",1br - 708ft2,4890.0,1.0,708.0,apartment,True,True,False,False,True,False,False,True,22


In [66]:
short.shape

(843731, 18)

In [67]:
short.dropna(subset=["price"], inplace=True)
short.shape

(838998, 18)

In [68]:
# short.dropna(inplace=True)
# short.shape

In [69]:
# short.drop_duplicates(inplace=True)
# short.shape

In [70]:
short['price_bucket'] = short['price'].apply(lambda x: f"{x//500:02.0f}x500")

In [71]:
short.head()

,latitude,longitude,district,address,housing,price,bedrooms,area,type,catsok,dogsok,garagea,garaged,furnished,laundryb,laundrys,wd,nthumbs,price_bucket
0,37.353754,-122.026428,sunnyvale,1249 Crescent Terrace,2br - 1300ft2,3150.0,2.0,1300.0,townhouse,False,False,False,True,False,False,False,True,18,06x500
1,38.008321,-121.864673,"Pittsburg, CA",2205 E Leland Rd,2br - 875ft2,1769.0,2.0,875.0,apartment,False,False,False,False,False,False,False,False,8,03x500
2,37.710526,-122.439706,excelsior / outer mission,375 Pope Street,2br - 1800ft2,3000.0,2.0,1800.0,house,False,False,True,False,False,True,False,False,5,06x500
3,38.274790,-122.019156,fairfield / vacaville,2436 Baltic Dr,3br - 1200ft2,2150.0,3.0,1200.0,house,False,False,False,True,False,False,False,False,10,04x500
4,37.671644,-122.076737,hayward / castro valley,1182 E Street near 2nd,1br - 726ft2,2150.0,1.0,726.0,apartment,True,True,False,False,False,False,False,True,8,04x500


In [72]:
short[short["price"] > 8000].shape

(10062, 19)

In [73]:
# short = short.drop(.index)
# df = df.drop(df[<some boolean condition>].index)
# short = short.drop(short[short["price"] > 8000].index)
short.shape

(838998, 19)

In [74]:
short["price_bucket"].describe()

count     838998
unique       158
top       04x500
freq      173534
Name: price_bucket, dtype: object

In [75]:
short["price_bucket"].unique()

array(['06x500', '03x500', '04x500', '08x500', '07x500', '05x500',
       '11x500', '02x500', '01x500', '35x500', '00x500', '13x500',
       '09x500', '12x500', '10x500', '17x500', '15x500', '19x500',
       '21x500', '14x500', '31x500', '23x500', '45x500', '90x500',
       '16x500', '20x500', '600000000000000000x500', '18x500', '24x500',
       '40x500', '27x500', '658x500', '80x500', '30x500', '29x500',
       '36x500', '28x500', '38x500', '22x500', '103x500', '25x500',
       '50x500', '32x500', '43x500', '44x500', '18482x500', '33x500',
       '49x500', '26x500', '37x500', '64x500', '62x500', '54x500',
       '56x500', '71x500', '39x500', '59x500', '70x500', '188x500',
       '34x500', '55x500', '60006x500', '100x500', '160x500', '88x500',
       '1380x500', '1400x500', '22222x500', '25002x500', '47404x500',
       '7550x500', '164x500', '53x500', '47x500', '83x500', '57x500',
       '48x500', '42606x500', '86009x500', '60x500', '1591x500',
       '102x500', '1589x500', '68x500', '

In [76]:
short.to_csv("data.csv", index=False) 

In [45]:
del short

In [53]:
maybe_pickle(prepare_data(df), "train.pd")

Pickling train.pd.
